In [1]:
import tensorflow as tf

a = tf.constant(1, name='a')
b = tf.constant(2, name='b')
c = tf.constant(3, name='c')
z = 2 * (a - b) + c
tf.print(f'Result: z = {z}')

Result: z = 1


In [2]:
def compute_z(a, b, c):
    r1 = tf.subtract(a, b)
    r2 = tf.multiply(2, r1)
    z = tf.add(r2, c)
    return z

In [3]:
tf.print(f'Scalar Inputs: {compute_z(1, 2, 3)}')

Scalar Inputs: 1


In [4]:
tf.print(f'Rank 1 Inputs: {compute_z([1], [2], [3])}')

Rank 1 Inputs: [1]


In [5]:
tf.print(f'Rank 2 Inputs: {compute_z([[1], [4]], [[2], [5]], [[3], [6]])}')

Rank 2 Inputs: [[1]
 [4]]


In [6]:
@tf.function
def compute_z(a, b, c):
    r1 = tf.subtract(a, b)
    r2 = tf.multiply(2, r1)
    z = tf.add(r2, c)
    return z

In [8]:
tf.print(f'Scalar Inputs: {compute_z(1, 2, 3)}')
tf.print(f'Rank 1 Inputs: {compute_z([1], [2], [3])}')
tf.print(f'Rank 2 Inputs:\n{compute_z([[1], [4]], [[2], [5]], [[3], [6]])}')

Scalar Inputs: 1
Rank 1 Inputs: [1]
Rank 2 Inputs:
[[1]
 [4]]


In [9]:
@tf.function(
    input_signature=(
        tf.TensorSpec(shape=[None], dtype=tf.int32),
        tf.TensorSpec(shape=[None], dtype=tf.int32),
        tf.TensorSpec(shape=[None], dtype=tf.int32)
    )
)
def compute_z(a, b, c):
    r1 = tf.subtract(a, b)
    r2 = tf.multiply(2, r1)
    z = tf.add(r2, c)
    return z

In [11]:
tf.print(f'Rank 1 Inputs: {compute_z([1, 4, 7], [2, 5, 8], [3, 6, 9])}')

Rank 1 Inputs: [1 4 7]


In [12]:
tf.print(f'Scalar Inputs: {compute_z(1, 2, 3)}')

ValueError: Python inputs incompatible with input_signature:
  inputs: (
    1,
    2,
    3)
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None),
    TensorSpec(shape=(None,), dtype=tf.int32, name=None),
    TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [13]:
a = tf.Variable(initial_value=3.14, name='var_a')
a

<tf.Variable 'var_a:0' shape=() dtype=float32, numpy=3.14>

In [14]:
b = tf.Variable(initial_value=[1, 2, 3], name='var_b')
b

<tf.Variable 'var_b:0' shape=(3,) dtype=int32, numpy=array([1, 2, 3])>

In [15]:
c = tf.Variable(initial_value=[True, False])
c

<tf.Variable 'Variable:0' shape=(2,) dtype=bool, numpy=array([ True, False])>

In [16]:
d = tf.Variable(initial_value=['aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'])
d

<tf.Variable 'Variable:0' shape=(1,) dtype=string, numpy=array([b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'], dtype=object)>

In [20]:
w = tf.Variable([1, 1, 4], trainable=False)
w.trainable

False

In [21]:
w

<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([1, 1, 4])>

In [22]:
w.assign([3, 1, 4], read_value=True)

<tf.Variable 'UnreadVariable' shape=(3,) dtype=int32, numpy=array([3, 1, 4])>

In [23]:
w.assign_add([2, -1, 2], read_value=False)

In [24]:
w

<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([5, 0, 6])>

In [25]:
tf.random.set_seed(1)
init = tf.keras.initializers.GlorotNormal()
tf.print(init(shape=(3, )))

[-0.722795904 1.01456821 0.251808226]


In [26]:
v = tf.Variable(init(shape=(2, 3)))
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 0.28982234, -0.7822928 , -0.0453659 ],
       [ 0.9609914 , -0.12000345,  0.7085282 ]], dtype=float32)>

In [28]:
class MyModule(tf.Module):
    def __init__(self):
        init = tf.keras.initializers.GlorotNormal()
        self.w1 = tf.Variable(init(shape=(2,3)), trainable=True)
        self.w2 = tf.Variable(init(shape=(1,2)), trainable=False)
        
m = MyModule()
print(f'All module variables: {[v.shape for v in m.variables]}')
print(f'Trainable variables: {[v.shape for v in m.trainable_variables]}')

All module variables: [TensorShape([2, 3]), TensorShape([1, 2])]
Trainable variables: [TensorShape([2, 3])]


In [29]:
tf.random.set_seed(1)
w = tf.Variable(tf.random.uniform((3, 3)))
w

<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32, numpy=
array([[0.16513085, 0.9014813 , 0.6309742 ],
       [0.4345461 , 0.29193902, 0.64250207],
       [0.9757855 , 0.43509948, 0.6601019 ]], dtype=float32)>

In [30]:
@tf.function
def compute_z(x):
    return tf.matmul(w, x)

x = tf.constant([[1], [2], [3]], dtype=tf.float32)
compute_z(x)

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[3.8610158],
       [2.9459305],
       [3.8262901]], dtype=float32)>

## GradientTape

In [31]:
w = tf.Variable(1.0)
b = tf.Variable(0.5)
w.trainable, b.trainable

(True, True)

In [35]:
x = tf.convert_to_tensor([1.4])
y = tf.convert_to_tensor([2.1])

with tf.GradientTape() as tape:
    z = tf.add(tf.multiply(w, x), b)
    loss = tf.reduce_sum(tf.square(y - z))

dloss_dw = tape.gradient(loss, w)    
tf.print(f'dL/dw: {dloss_dw}')

dL/dw: -0.5599997639656067


In [36]:
with tf.GradientTape() as tape:
    tape.watch(x)
    z = tf.add(tf.multiply(w, x), b)
    loss = tf.reduce_sum(tf.square(y - z))
    
dloss_dx = tape.gradient(loss, x)
tf.print(f'dL/dx: {dloss_dx}')

dL/dx: [-0.39999986]


In [39]:
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    z = tf.add(tf.multiply(w, x), b)
    loss = tf.reduce_sum(tf.square(y - z))
    
dloss_dw = tape.gradient(loss, w)    
tf.print(f'dL/dw: {dloss_dw}')

dloss_dx = tape.gradient(loss, x)
tf.print(f'dL/dx: {dloss_dx}')

dloss_db = tape.gradient(loss, b)
tf.print(f'dL/db: {dloss_db}')

dL/dw: -0.5599997639656067
dL/dx: [-0.39999986]
dL/db: -0.39999985694885254


In [40]:
optimizer = tf.keras.optimizers.SGD()
optimizer.apply_gradients(zip([dloss_dw, dloss_db], [w, b]))

tf.print('Updated w:', w)
tf.print('Updated b:', b)

Updated w: 1.0056
Updated b: 0.504


## Keras API